## Create Tables in Data Base

Tutorial [Link](https://colab.research.google.com/github/GoogleCloudPlatform/cloud-sql-python-connector/blob/main/samples/notebooks/mysql_python_connector.ipynb#scrollTo=wfEvH386zX2V)

In [2]:
# !gcloud sql databases create kexpDB --instance=kexp-sandbox

In [14]:
import google.auth

credentials, project_id = google.auth.default()

# install dependencies
# pip install cloud-sql-python-connector["pymysql"] SQLAlchemy==2.0.7

import sys
if '..' not in sys.path:
    sys.path.append('..')

from scripts.utils import openfile
from scripts.utils import Hosts, Plays, Programs, Shows, Timeslots


from google.cloud.sql.connector import Connector

import sqlalchemy
from sqlalchemy import MetaData
from sqlalchemy.orm import sessionmaker
from sqlalchemy.sql.expression import func

from creds import user

import requests
import pandas as pd

In [2]:
# initialize parameters
INSTANCE_CONNECTION_NAME = user['INSTANCE_CONNECTION_NAME']# i.e demo-project:us-central1:demo-instance
print(f"Your instance connection name is: {INSTANCE_CONNECTION_NAME}")
DB_USER = user['DB_USER']
DB_PASS = user['DB_PASS']
DB_NAME = user['DB_NAME']


root_path = user['root_path']

Your instance connection name is: kexp-explore:us-central1:kexp-sandbox


In [3]:


# initialize Connector object
connector = Connector()

# function to return the database connection object
def getconn():
    conn = connector.connect(
        INSTANCE_CONNECTION_NAME,
        "pymysql",
        user=DB_USER,
        password=DB_PASS,
        db=DB_NAME
    )
    return conn

# create connection pool with 'creator' argument to our connection object function
pool = sqlalchemy.create_engine(
    "mysql+pymysql://",
    creator=getconn,
)

engine = pool

In [4]:
# Create a MetaData object
metadata = MetaData()

# Reflect the existing database schema into the MetaData object
metadata.reflect(bind=engine)

# Access information about tables in the 'tables' dictionary attribute
for table_name, table in metadata.tables.items():
    print(f"Table name: {table_name}")
    # Access columns, constraints, etc. through the 'table' object


Table name: hosts
Table name: plays
Table name: programs
Table name: shows
Table name: timeslots


In [5]:
metadata.tables.items()

dict_items([('hosts', Table('hosts', MetaData(), Column('id', INTEGER(), table=<hosts>, primary_key=True, nullable=False), Column('uri', VARCHAR(length=255), table=<hosts>, nullable=False), Column('name', VARCHAR(length=100), table=<hosts>, nullable=False), Column('image_uri', VARCHAR(length=255), table=<hosts>), Column('thumbnail_uri', VARCHAR(length=255), table=<hosts>), Column('is_active', TINYINT(display_width=1), table=<hosts>, nullable=False), schema=None)), ('plays', Table('plays', MetaData(), Column('id', INTEGER(), table=<plays>, primary_key=True, nullable=False), Column('uri', VARCHAR(length=255), table=<plays>, nullable=False), Column('airdate', TIMESTAMP(), table=<plays>, nullable=False), Column('show_id', INTEGER(), table=<plays>, nullable=False), Column('song', VARCHAR(length=255), table=<plays>, nullable=False), Column('track_id', VARCHAR(length=255), table=<plays>, nullable=False), Column('recording_id', VARCHAR(length=255), table=<plays>, nullable=False), Column('artis

### GreenField Configuration

In [6]:


# connect to connection pool
with pool.connect() as db_conn:
  # create hosts table in our kexp-sandbox database
  db_conn.execute(sqlalchemy.text(openfile(root_path+'hosts.sql')))

  # commit transaction (SQLAlchemy v2.X.X is commit as you go)
  db_conn.commit()

  # create plays table in our kexp-sandbox database
  db_conn.execute(sqlalchemy.text(openfile(root_path+'plays.sql')))

  # commit transaction (SQLAlchemy v2.X.X is commit as you go)
  db_conn.commit()

  # create programs table in our kexp-sandbox database
  db_conn.execute(sqlalchemy.text(openfile(root_path+'programs.sql')))

  # commit transaction (SQLAlchemy v2.X.X is commit as you go)
  db_conn.commit()

  # create shows table in our kexp-sandbox database
  db_conn.execute(sqlalchemy.text(openfile(root_path+'shows.sql')))

  # commit transaction (SQLAlchemy v2.X.X is commit as you go)
  db_conn.commit()

  # create timeslots table in our kexp-sandbox database
  db_conn.execute(sqlalchemy.text(openfile(root_path+'timeslots.sql')))

  # commit transaction (SQLAlchemy v2.X.X is commit as you go)
  db_conn.commit()


File contents:
CREATE TABLE IF NOT EXISTS hosts (
                id INT PRIMARY KEY,
                uri VARCHAR(255) UNIQUE NOT NULL,
                name VARCHAR(100) UNIQUE NOT NULL,
                image_uri VARCHAR(255),
                thumbnail_uri VARCHAR(255),
                is_active BOOL NOT NULL
            );
File contents:
CREATE TABLE IF NOT EXISTS plays (
                id INT PRIMARY KEY,
                uri VARCHAR(255) UNIQUE NOT NULL,
                airdate TIMESTAMP NOT NULL,
                show_id INT NOT NULL,
                song VARCHAR(255) NOT NULL,
                track_id VARCHAR(255) NOT NULL,
                recording_id VARCHAR(255) NOT NULL,
                artist VARCHAR(255) NOT NULL,
                album VARCHAR(255) NOT NULL,
                release_id VARCHAR(255),
                release_group_id VARCHAR(255),
                release_date VARCHAR(255),
                is_local BOOL,
                is_request BOOL,
                is_live BO

In [7]:
# Reflect the existing database schema into the MetaData object
metadata.reflect(bind=engine)

for t in metadata.tables.items():
    print(t[0])

hosts
plays
programs
shows
timeslots


## Populate Tables

In [8]:
r = requests.get('https://api.kexp.org/v2/')
r.status_code

urls = r.json()
urls

{'hosts': 'https://api.kexp.org/v2/hosts/',
 'programs': 'https://api.kexp.org/v2/programs/',
 'shows': 'https://api.kexp.org/v2/shows/',
 'plays': 'https://api.kexp.org/v2/plays/',
 'timeslots': 'https://api.kexp.org/v2/timeslots/'}

### Hosts Populate

In [11]:
hosts = requests.get('https://api.kexp.org/v2/hosts')
hosts_json=hosts.json()
print(f"Total Entries: {hosts_json['count']}")


Total Entries: 98


In [12]:
    
# Create a session
Session = sessionmaker(bind=engine)
session = Session()
    
# Insert data into the table
try:
    for data in hosts_json['results']:
        user = Hosts(**data)
        session.add(user)

    # Commit the transaction
    session.commit()
    print("Data inserted successfully!")

except Exception as e:
    # Roll back the transaction if an error occurs
    session.rollback()
    print(f"Error: {e}")

finally:
    # Close the session
    session.close()


Data inserted successfully!


In [16]:
# Query the table for 5 random rows
try:
    random_rows = session.query(Hosts).order_by(func.rand()).limit(5).all()
    for row in random_rows:
        print(row.id, row.uri, row.name, row.image_uri, row.thumbnail_uri, row.is_active)

except Exception as e:
    print(f"Error: {e}")

finally:
    # Close the session
    session.close()

7 https://api.kexp.org/v2/hosts/7/ Diane Horn https://www.kexp.org/filer/canonical/1529969977/10632/  False
9 https://api.kexp.org/v2/hosts/9/ DJ El Toro https://kexpstorage.blob.core.windows.net/images/hosts/111335  False
58 https://api.kexp.org/v2/hosts/58/ B-Mello   False
8 https://api.kexp.org/v2/hosts/8/ DJ Alex https://www.kexp.org/filer/canonical/1529969764/10631/  True
3 https://api.kexp.org/v2/hosts/3/ Brian Foss https://www.kexp.org/filer/canonical/1529969446/10628/  True


In [17]:

while hosts_json['next'] != None:
    hosts = requests.get(hosts_json['next'])
    hosts_json=hosts.json()
    # Create a session
    Session = sessionmaker(bind=engine)
    session = Session()
        
    # Insert data into the table
    try:
        for data in hosts_json['results']:
            user = Hosts(**data)
            session.add(user)

        # Commit the transaction
        session.commit()
        print("Data inserted successfully!")

    except Exception as e:
        # Roll back the transaction if an error occurs
        session.rollback()
        print(f"Error: {e}")

    finally:
        # Close the session
        session.close()

Data inserted successfully!
Data inserted successfully!
Data inserted successfully!
Data inserted successfully!


### Programs Populate

In [19]:
programs = requests.get(urls['programs'])
programs_json=programs.json()
print(f"Total Entries: {programs_json['count']}")

Total Entries: 40


In [20]:
programs_json

{'count': 40,
 'next': 'https://api.kexp.org/v2/programs/?limit=20&offset=20',
 'previous': None,
 'results': [{'id': 26,
   'uri': 'https://api.kexp.org/v2/programs/26/',
   'name': '90.TEEN',
   'description': 'Rock,Eclectic,Variety Mix',
   'tags': 'Rock,Eclectic,Variety Mix',
   'is_active': True},
  {'id': 38,
   'uri': 'https://api.kexp.org/v2/programs/38/',
   'name': 'Astral Plane',
   'description': '',
   'tags': 'Rock,Eclectic,Psychedelic',
   'is_active': True},
  {'id': 1,
   'uri': 'https://api.kexp.org/v2/programs/1/',
   'name': 'Audioasis',
   'description': "Audioasis is KEXP's long-running Northwest music show hosted by Sharlese Metcalf. Past hosts have included Jonathen Poneman, Jason Hughes, Scott Vanderpool, Stevie Zoom, Sean Nelson and Hannah Levin. John Richards helps produce the show. Audioasis delivers three hours of local and live bands from all areas and genres of the Northwest. On Audioasis, you'll hear the new, the old, the demos, the vinyl, and the future

In [22]:
# Create a session
Session = sessionmaker(bind=engine)
session = Session()
    
# Insert data into the table
try:
    for data in programs_json['results']:
        program = Programs(**data)
        session.add(program)

    # Commit the transaction
    session.commit()
    print("Data inserted successfully!")

except Exception as e:
    # Roll back the transaction if an error occurs
    session.rollback()
    print(f"Error: {e}")

finally:
    # Close the session
    session.close()

Data inserted successfully!


In [24]:
# Create a session
Session = sessionmaker(bind=engine)
session = Session()
# Query the table for 5 random rows
try:
    random_rows = session.query(Programs).order_by(func.rand()).limit(5).all()
    for row in random_rows:
        print(row.id, row.uri, row.name, row.description, row.tags, row.is_active)

except Exception as e:
    print(f"Error: {e}")

finally:
    # Close the session
    session.close()

37 https://api.kexp.org/v2/programs/37/ Eastern Echoes  Eclectic,Variety,World True
7 https://api.kexp.org/v2/programs/7/ Positive Vibrations Placing equal legitimacy upon all styles of Jamaican musical expression, Positive Vibrations joins the dots between Ska, Rocksteady, Roots Reggae, Dub, and Dancehall. Kid Hops educates as he entertains - riding rhythms across artists, labels, and decades, he demystifies the connections between producers and studios. From the roughest riddims, to the sweetest harmonies and rudest of rudebwoy deejays, Positive Vibrations provides the perfect blend of Jamaican sounds. Reggae,World True
22 https://api.kexp.org/v2/programs/22/ Friday Night Electronic, Soul, R&B, Hip-Hop, Rock Rock,Eclectic,Variety Mix True
35 https://api.kexp.org/v2/programs/35/ Overnight Afrobeats  Afrobeat False
34 https://api.kexp.org/v2/programs/34/ Mechanical Breakdown  Wave,Synth,Post Punk,EBM True


In [26]:
while programs_json['next'] != None:
    programs = requests.get(programs_json['next'])
    hosts_json=programs.json()
    # Create a session
    Session = sessionmaker(bind=engine)
    session = Session()
        
    # Insert data into the table
    try:
        for data in hosts_json['results']:
            user = Hosts(**data)
            session.add(user)

        # Commit the transaction
        session.commit()
        print("Data inserted successfully!")

    except Exception as e:
        # Roll back the transaction if an error occurs
        session.rollback()
        print(f"Error: {e}")

    finally:
        # Close the session
        session.close()

Error: 'description' is an invalid keyword argument for Hosts
Error: 'description' is an invalid keyword argument for Hosts
Error: 'description' is an invalid keyword argument for Hosts
Error: 'description' is an invalid keyword argument for Hosts
Error: 'description' is an invalid keyword argument for Hosts
Error: 'description' is an invalid keyword argument for Hosts
Error: 'description' is an invalid keyword argument for Hosts
Error: 'description' is an invalid keyword argument for Hosts
Error: 'description' is an invalid keyword argument for Hosts
Error: 'description' is an invalid keyword argument for Hosts
Error: 'description' is an invalid keyword argument for Hosts
Error: 'description' is an invalid keyword argument for Hosts
Error: 'description' is an invalid keyword argument for Hosts
Error: 'description' is an invalid keyword argument for Hosts
Error: 'description' is an invalid keyword argument for Hosts
Error: 'description' is an invalid keyword argument for Hosts
Error: '

KeyboardInterrupt: 

## Delete Table

In [ ]:
# connect to connection pool
with pool.connect() as db_conn:
  # delete plays table in our kexp-sandbox database
  db_conn.execute(sqlalchemy.text('DROP TABLE plays'))

  # commit transaction (SQLAlchemy v2.X.X is commit as you go)
  db_conn.commit()

  # create plays table in our kexp-sandbox database
  db_conn.execute(sqlalchemy.text(openfile(root_path+'plays.sql')))

  # commit transaction (SQLAlchemy v2.X.X is commit as you go)
  db_conn.commit()